In [42]:
import torch.utils.data as Data
from datasets import load_dataset
import random
from transformers import AutoTokenizer, DataCollatorForLanguageModeling

In [43]:
train_dataset = load_dataset(path='dair-ai/emotion')['train']
train_dataset[:10]

Using the latest cached version of the dataset since dair-ai/emotion couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'split' at C:\Users\duanm\.cache\huggingface\datasets\dair-ai___emotion\split\0.0.0\cab853a1dbdf4c42c2b3ef2173804746df8825fe (last modified on Wed Jun 18 11:28:24 2025).


{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'im grabbing a minute to post i feel greedy wrong',
  'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
  'i am feeling grouchy',
  'ive been feeling a little burdened lately wasnt sure why that was',
  'ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny',
  'i feel as confused about life as a teenager or as jaded as a year old man',
  'i have been with petronas for years i feel that petronas has performed well and made a huge profit',
  'i feel romantic too'],
 'label': [0, 0, 3, 2, 3, 0, 5, 4, 1, 2]}

In [44]:
tokenizer = AutoTokenizer.from_pretrained('E:\huggingface_models\Qwen3-0.6B')
train_dataset = train_dataset.map(lambda samples: tokenizer(samples["text"]), batched=True)
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 16000
})

In [45]:
for i in train_dataset.select(range(0, 10)):
    print(len(i['input_ids']))

5
21
10
18
5
14
24
18
23
4


In [46]:
batch_size = 4

In [47]:
train_dataset = train_dataset.remove_columns('text')
yy_list = []
for i in range(0, len(train_dataset)):
    if i % batch_size == 0:
        length =  random.randint(32, 64)
    yy_list.append(list(range(0, length)))
train_dataset = train_dataset.add_column('yy', yy_list)
train_dataset

Dataset({
    features: ['label', 'input_ids', 'attention_mask', 'yy'],
    num_rows: 16000
})

In [ ]:
dac = Data.DataLoader(train_dataset, 
                      # 保留所有字段(所有字段必须为数值类型)
                      # tokenizer类型字段(如:input_ids,attention_mask)动态填充到最大长度
                      # 其他字段每个batch_size的数据必须可以转换为一个tensor
                      collate_fn=DataCollatorForLanguageModeling(tokenizer, 
                                                                 # Whether or not to use masked language modeling. If set to False, the labels are the same as the inputs with the padding tokens ignored (by setting them to -100). Otherwise, the labels are -100 for non-masked tokens and the value to predict for the masked token.
                                                                 mlm=False), 
                      batch_size=batch_size)

for i in dac:
    print(i.keys())
    print("yy: ", i["yy"].shape)
    print("label: ", i['label'])

    # # labels内部计算实现(mlm=False时):
    # labels = batch["input_ids"].clone()
    # if self.tokenizer.pad_token_id is not None:
    #     labels[labels == self.tokenizer.pad_token_id] = -100
    # batch["labels"] = labels
    print("labels: ", i['labels'])
    print("labels.shape: ", i['labels'].shape)
    print('input_ids: ', i['input_ids'])
    print("input_ids.shape: ", i['input_ids'].shape)
    print("attention_mask.shape: ", i['attention_mask'].shape)
    break

dict_keys(['label', 'input_ids', 'attention_mask', 'yy', 'labels'])
yy:  torch.Size([4, 50])
label:  tensor([0, 0, 3, 2])
labels:  tensor([[   72, 47607,  2666,  2784, 53773,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100],
        [   72,   646,   728,   504,  8266,   773, 74223,   311,   773, 67365,
         37550,  1101,   504,  1660,  2163,  4325,   879, 33572,   323,   374,
         34347],
        [  318, 48930,   264,  9383,   311,  1736,   600,  2666, 55980,  4969,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100],
        [   72,  1079,  3512,  8266, 82274,   911,   279, 39411,   600,   686,
          1414,   429,   432,   374,  2058,   389,   279,  3343,  -100,  -100,
          -100]])
labels.shape:  torch.Size([4, 21])
input_ids:  tensor([[    72,  47607,   2666,   2784,  53773, 151643, 151643, 151643, 151643,
         151643, 151643, 151643, 151643,